In [20]:
# chapter 1 
from bs4 import BeautifulSoup
import requests
import re

In [80]:
def removePageTags(html):
    soup = BeautifulSoup(html, "html.parser")
    for data in soup(['style', 'script']):
        data.decompose()
    return ' '.join(soup.stripped_strings)

def getChapterNumbers(part):
    chapterNumberList = []
    partUrl = "https://uscode.house.gov/view.xhtml?req=granuleid%3AUSC-prelim-title18-part" + str(part) + "&saved=L3ByZWxpbUB0aXRsZTE4L3BhcnQxL2NoYXB0ZXIx%7CZ3JhbnVsZWlkOlVTQy1wcmVsaW0tdGl0bGUxOC1jaGFwdGVyMQ%3D%3D%7C%7C%7C0%7Cfalse%7Cprelim&edition=prelim"
    partHtml = requests.get(partUrl)
    soup = BeautifulSoup(partHtml.content, "html.parser")
    chapterTags = soup.find_all("h3", class_="chapter-head")
    for chapter in chapterTags:
        chapterNumber = re.findall(r'\d+[a-zA-Z]?', chapter.text)[0]
        chapterNumberList.append(chapterNumber)
    return chapterNumberList
        

def getChapterSectionsTitles(soup):
    chapterSectionTitles = []
    titleTags = soup.find_all("h3", class_="section-head")
    for title in titleTags:
        if not ("repealed" in title.text.lower()):
            chapterSectionTitles.append(title.text)
    return chapterSectionTitles 

def sectionInformation(chapterNumber, sectionNumber, legalSectionInformation):
    sectionUrl = "https://uscode.house.gov/view.xhtml?hl=false&edition=prelim&path=%2Fprelim%40title18%2Fpart1%2Fchapter" + str(chapterNumber) + "&req=granuleid%3AUSC-prelim-title18-section" + str(sectionNumber) + "&num=0&saved=L3ByZWxpbUB0aXRsZTE4L3BhcnQxL2NoYXB0ZXIx%7CZ3JhbnVsZWlkOlVTQy1wcmVsaW0tdGl0bGUxOC1jaGFwdGVyMQ%3D%3D%7C%7C%7C0%7Cfalse%7Cprelim"
    sectionHtml = requests.get(sectionUrl, timeout=20)
    soup = BeautifulSoup(sectionHtml.content, "html.parser")
    sectionTitle = soup.title.string
    sectionDescription = soup.find_all("p", class_=re.compile(r'statutory-body'))
    legalSectionDescription = ""
    for item in sectionDescription:
        legalSectionDescription += item.text
        legalSectionDescription += "\n"
    legalSectionInformation[sectionTitle] = legalSectionDescription

def getRelevantChapterContent(chapterNumber, legalSectionInformation):
    chapterUrl = "https://uscode.house.gov/view.xhtml?path=/prelim@title18/part1/chapter" + str(chapterNumber) + "&edition=prelim"
    chapterHtml = requests.get(chapterUrl, timeout=20)
    soup = BeautifulSoup(chapterHtml.content, "html.parser")
    chapterSectionTitles = getChapterSectionsTitles(soup)
    for sectionTitle in chapterSectionTitles:
        sectionNumber = re.findall(r'\d+[a-zA-Z]?', sectionTitle)[0]
        sectionInformation(chapterNumber, sectionNumber, legalSectionInformation)

In [84]:
legalSectionInformation = {}
chapterNumbers = getChapterNumbers(1)
for i in chapterNumbers:
    print(f"In Chapter: {i}")
    getRelevantChapterContent(i, legalSectionInformation)

In Chapter: 1
In Chapter: 2
In Chapter: 3
In Chapter: 5
In Chapter: 7
In Chapter: 9
In Chapter: 10
In Chapter: 11
In Chapter: 11A
In Chapter: 11B
In Chapter: 12
In Chapter: 13
In Chapter: 15
In Chapter: 17
In Chapter: 17A
In Chapter: 18
In Chapter: 19
In Chapter: 21
In Chapter: 23
In Chapter: 25
In Chapter: 26
In Chapter: 27
In Chapter: 29
In Chapter: 31
In Chapter: 33
In Chapter: 35
In Chapter: 37
In Chapter: 39
In Chapter: 40
In Chapter: 41
In Chapter: 42
In Chapter: 43
In Chapter: 44
In Chapter: 45
In Chapter: 46
In Chapter: 47
In Chapter: 49
In Chapter: 50
In Chapter: 50A
In Chapter: 51
In Chapter: 53
In Chapter: 55
In Chapter: 57
In Chapter: 59
In Chapter: 61
In Chapter: 63
In Chapter: 65
In Chapter: 67
In Chapter: 68
In Chapter: 69
In Chapter: 71
In Chapter: 73
In Chapter: 74
In Chapter: 75
In Chapter: 77
In Chapter: 79
In Chapter: 81
In Chapter: 83
In Chapter: 84
In Chapter: 85
In Chapter: 87
In Chapter: 88
In Chapter: 89
In Chapter: 90
In Chapter: 90A
In Chapter: 91
In Chapter:

In [88]:
file_path = 'part1Content.txt'
with open(file_path, 'w') as file:
    for key, value in legalSectionInformation.items():
        file.write(f"{key}:\n{value}\n")